In [11]:
import pandas as pd
import json as json

In [12]:
def loadFiles(version, arena):
    file_path     = "../data/"
    list_of_files = []
    arena_list    = ["arena-1", "arena-2", "arena-3", "arena-4", "arena-5", "arena-6", "arena-7", "arena-8", "arena-9",]
    file_path = getVersionPath(version, file_path)
    if arena in arena_list:   
        file_path        += "/" + arena + "/"
        encounter_pokemon = file_path + "EncounterablePokemon.csv"
        pokemon_of_leader = file_path + "LeaderPokemon.csv"
        arena_leader = file_path + "ArenaLeader.csv"
        move_of_in        = file_path + "MovesOfInterest.csv"
        list_of_files.append(encounter_pokemon)
        list_of_files.append(pokemon_of_leader)
        list_of_files.append(arena_leader)
        list_of_files.append(move_of_in)  
    else:
        print("[-] Arena not found...")
    return list_of_files

def getVersionPath(version, file_path):
    if version == "red":
        file_path += "red"
        return file_path
    elif version == "blue":
        file_path += "blue"
        return file_path

In [1]:
def loadStones(arena):
    #["moon-stone", "fire-stone", "thunder-stone", "water-stone", "leaf-stone"]
    ## TODO: add arena stone data
    list_of_stones = []
    
    arena1_stones = []
    arena2_stones = arena1_stones + ["moon-stone"]
    arena3_stones = arena2_stones + ["fire-stone", "thunder-stone" , "water-stone", "leaf-stone" ]
    
    if arena == "arena-1":
        pass
    elif arena == "arena-2":
        for stone in arena2_stones:  
            list_of_stones.append(stone)
    elif arena == "arena-3":
        for stone in arena3_stones:
            list_of_stones.append(stone)
    return list_of_stones

In [14]:
# extracts all data of interest for the enemy leader pokemon
def extractPokemonStatsFromTable(pokemonName):
    pokeRow = pokemonTable.loc[pokemonTable['Name'] == pokemonName]
    return pokeRow

In [46]:
# STATIC CSV FILES
pokemonTable = pd.read_csv('../Data/Pokemon.csv')
pokemonTypeTable = pd.read_csv('../Data/PokemonTypeChart.csv')

# SETUP ARENA-1
ARENA = 'arena-8'
GAME = 'red'
csvPaths = loadFiles(GAME, ARENA)

encounterable_pokemonTable = pd.read_csv(csvPaths[0])

# [name, moves, encounter, originIfEvo, movesByVM, movesByTM]
encounterable_pokemonParsed = []

for row in encounterable_pokemonTable.values.tolist():
    row = row[1:]
    moves = row[1]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[1] = moves
    
    VMmoves = row[4]
    VMmoves = VMmoves.replace("'", "\"") 
    VMmoves = json.loads(VMmoves)
    row[4] = VMmoves
    
    TMmoves = row[5]
    TMmoves = TMmoves.replace("'", "\"") 
    TMmoves = json.loads(TMmoves)
    row[5] = TMmoves
   
    encounterable_pokemonParsed.append(row)
    
leader_pokemon =  pd.read_csv(csvPaths[1])

## um moves von leaderPokemon zu parsen
#test = leader_pokemon.loc[leader_pokemon['name']=='geodude']['moves'].values.tolist()[0].replace("'", "\"") 
#json.loads(test)

arena_leader = pd.read_csv(csvPaths[2])
arena_leader_parsed = []
for row in arena_leader.values.tolist(): 
    moves = row[3]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[3] = moves
    arena_leader_parsed.append(row[1:])

moves_of_interest = pd.read_csv(csvPaths[3])

## extracted Variables from DATAFRAME
AVERAGE_LEVEL = arena_leader['averageLevel'][0]

## Variables for each Arena
arena1_Options = loadStones(ARENA)

## get leaderPokemonData 
# [stats, moves]
leaderPokemonData = {}
for leaderPokemon in arena_leader_parsed[0][2]:
    moves = leader_pokemon.loc[leader_pokemon['name']==leaderPokemon]['moves'].values.tolist()[0].replace("'", "\"") 
    moves = json.loads(moves)
    moveDict = {}
    for move in moves:
        moveData = moves_of_interest.loc[moves_of_interest['name']==move]
        moveDict[move] = moveData
    leaderPokemonData[leaderPokemon] = [ extractPokemonStatsFromTable(leaderPokemon), moveDict] 


In [16]:
# spuckt liste von Pokemon aus welche zur verf. stehen

# @para1: encounterable_pokemon - liste von antreffbaren pokemon :: [index,name,encounterOrEvolutionTrigger,...]
# @para2: averageLevel - averageLevel des ArenaLeiters
# @para3: stones_available - Die Entw. Steine welche zur verfuegung stehen
# @return: poss_team_member - liste von pokemon welche mit dem averageLevel und den vorhanden Steinen erreichbar sind :: [index,name,encounterOrEvolutionTrigger,...]
def possPokemonArenaNLeaderM(encounterable_pokemon, averageLevel, stones_available):
    
    encounterable_pokemon_available = []
    for pokemon in encounterable_pokemon:
        encounterOrEvolutionTrigger = pokemon[2]
        if encounterOrEvolutionTrigger.isnumeric() and int(encounterOrEvolutionTrigger) <= averageLevel:
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger == 'walk' or encounterOrEvolutionTrigger == 'starter' or encounterOrEvolutionTrigger == 'special' or encounterOrEvolutionTrigger == 'trade':
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger in stones_available:
            encounterable_pokemon_available.append(pokemon)
    return encounterable_pokemon_available

In [17]:
def extractMoveData(moveName):
    return moves_of_interest.loc[moves_of_interest['name']==moveName]

#  extractMoveData('gust')

# returns moveTable row

In [18]:
def getMoveData(pokemonMoveNames):
    moveData = []
    for move in pokemonMoveNames:
        moveData.append(extractMoveData(move))
    return moveData

# returns array of moveTable rows

In [19]:
# extracts available move names from pokemon list item

# @param ourPokemonAndMoves:
#    [ 
#       index 0: 'name',
#       index 1: [{name:'moveName', levelLearnedAt: x}]
#       index 2: encounterOrEvoTrigger -> nur 1 starter pokemon erlaubt
#       index 3: origin if evolution -> nur 1 starter evolution erlaubt
#       index 4: movesByTM
#       index 5: movesByVM
#    ]

# @param averageLevel: int -> average LeaderPokemon level


def calcAvailableMoveNamesForPokemon(ourPokemonAndMoves, averageLevel):
    moves = []
    # LEVEL UP MOVES
    for move in ourPokemonAndMoves[1]:
        if move['levelLearnedAt'] <= averageLevel:
            moves.append(move['name'])
    # VM MOVES
    for move in ourPokemonAndMoves[4]:
        moves.append(move)
    # TM MOVES
    for move in ourPokemonAndMoves[5]:
        moves.append(move)
    return moves

# returns array with moveNames:  ['gust', 'sand-attack', 'quick-attack']

# calcAvailableMoveNamesForPokemon(possiblePokemon[0], 13)

In [20]:
def retreiveMoveEffectiveness(moveType, vsMoveType):
    return pokemonTypeTable.loc[pokemonTypeTable['type'] == moveType][vsMoveType].values[0]

# returns the effectiveness value from kaggleTable
# 0.5 | 1 | 2

In [21]:
## THIS FUNCTION IS ENTRY POINT AFTER SETTINGS

def createAvailablePokemonWithAvailableMoves(encounterable_pokemonParsed, AVERAGE_LEVEL, arena1_Options):
    pokeObjects = {}
    availablePokemonArrays = possPokemonArenaNLeaderM(encounterable_pokemonParsed, AVERAGE_LEVEL, arena1_Options)
    
    for pokemonArrayElement in availablePokemonArrays: 
        name = pokemonArrayElement[0]
        pokemonTableRow = extractPokemonStatsFromTable(name)
        availableMoves = calcAvailableMoveNamesForPokemon(pokemonArrayElement, AVERAGE_LEVEL)
        availableMoveDetails = getMoveData(availableMoves)
        pokeObjects[name] = {}
        pokeObjects[name]['name'] = name
        pokeObjects[name]['stats'] = pokemonTableRow
        pokeObjects[name]['moves'] = availableMoveDetails
        pokeObjects[name]['origin'] = pokemonArrayElement[2]
        pokeObjects[name]['originIfEvo'] = pokemonArrayElement[3]
        pokeObjects[name]['movesVs'] = {}
        pokeObjects[name]['scoreVs'] = {}
        pokeObjects[name]['actualMovesVs'] = {}
        
    return pokeObjects

# returns { pokemonName: {stats: pokeStatTableRow, moves: availableMovesTableRow}}

In [22]:
def calcEffectiveMovesVs(pokemonObject, theirPoke):
    theirType1 = theirPoke[0]['Type 1'].values[0].lower()

    theirType2 = theirPoke[0]['Type 2'].values[0]
    
    theirAtk = theirPoke[0]['Attack'].values[0]
    theirDefense = theirPoke[0]['Defense'].values[0]
    theirSpAtk = theirPoke[0]['Sp. Atk'].values[0]
    theirSpDefense = theirPoke[0]['Sp. Def'].values[0]
    
    ourAtk = pokemonObject['stats']['Attack'].values[0]
    ourDefense = pokemonObject['stats']['Defense'].values[0]
    ourSpAtk = pokemonObject['stats']['Sp. Atk'].values[0]
    ourSpDefense = pokemonObject['stats']['Sp. Def'].values[0]
    
    if isinstance(theirType2, float):
        pass
    else:
        theirType2 = theirType2.lower()
    effectivenessList = []
    
    ourPokeMoves = pokemonObject['moves']
    for move in ourPokeMoves:
        if len(move['moveType'].values) > 0:
            moveType = move['moveType'].values[0]
            #
            damageClass = move['damageClass'].values[0]
            power = move['power'].values[0]
            movePP = move['pp'].values[0]
            movePrio = move['priority'].values[0]
            moveAccuracy = move['accuracy'].values[0]
            #
            
            moveName =  move['name'].values[0]
            effective = retreiveMoveEffectiveness(moveType, theirType1)
            atkVsDefense = 0
            if(damageClass == 'status'):
                atkVsDefense = 1.0
                continue
            elif(damageClass == 'special'):
                atkVsDefense = ourSpAtk / theirSpDefense
            elif(damageClass == 'physical'):
                atkVsDefense = ourAtk / theirAtk
            effectivenessList.append({'name':moveName, 'effective':effective, 'atkVsDefense':atkVsDefense, 'moveType': moveType, 'damageClass': damageClass, 'power':power, 'movePP': movePP, 'movePrio': movePrio, 'moveAccuracy': moveAccuracy })
        else:
            pass
    effectivenessList = sorted(effectivenessList, key=lambda d: d['effective'],reverse=True) 
    return(effectivenessList)

In [23]:
## for each arenaLeaderPokemon append the moveEffectiveness to the pokemonObjects

def appendMovesVsToPokemonObject(availablePokemonObjects, leaderPokemonData):
    for key in availablePokemonObjects:
        pokemonObject = availablePokemonObjects[key]
        for leaderPokeKey in leaderPokemonData:
            leaderPokemon = leaderPokemonData[leaderPokeKey]
            pokemonObject['movesVs'][leaderPokeKey] = calcEffectiveMovesVs(pokemonObject, leaderPokemon)

In [32]:
def appendMoveScoreToPokemonObject(ourAvailablePokemonObjects, leaderPokemonData):
     for ourName in ourAvailablePokemonObjects:
            ourPokeObject = ourAvailablePokemonObjects[ourName]
            for theirName in leaderPokemonData:
                theirPoke = leaderPokemonData[theirName]
                i = 0
                score = 0
                
                highestScoreVs = 0
                lowestScoreVs = 0
                
                actualMovesVs = []
                
                for move in ourPokeObject['movesVs'][theirName]:
                    if(i<4):
                        score += move['effective'] * move['atkVsDefense']
                        actualMovesVs.append(move['name'])
                        i += 1
                    else:
                        break
                        
                        
                theirAtk = theirPoke[0]['Attack'].values[0]
                theirDefense = theirPoke[0]['Defense'].values[0]
                theirSpAtk = theirPoke[0]['Sp. Atk'].values[0]
                theirSpDefense = theirPoke[0]['Sp. Def'].values[0]
                
                ourAtk = ourPokeObject['stats']['Attack'].values[0]
                ourDefense = ourPokeObject['stats']['Defense'].values[0]
                ourSpAtk = ourPokeObject['stats']['Sp. Atk'].values[0]
                ourSpDefense = ourPokeObject['stats']['Sp. Def'].values[0]
                
                for arenaPmove in theirPoke[1]:
                    ourPokeType1 = ourPokeObject['stats']['Type 1'].values[0].lower()
                    theirMoveType = theirPoke[1][arenaPmove]['moveType'].values[0]
                    theirMoveDamageClass = theirPoke[1][arenaPmove]['damageClass'].values[0]
                    
                    atkVsDefense = 0
                    if(theirMoveDamageClass == 'status'):
                        continue
                        effective = 1.0
                        atkVsDefense = 1.0
                    elif(theirMoveDamageClass == 'special'):
                        atkVsDefense = theirSpAtk / ourSpDefense
                    elif(theirMoveDamageClass == 'physical'):
                        atkVsDefense = theirAtk / ourAtk
                    
                    score -= retreiveMoveEffectiveness(theirMoveType, ourPokeType1) * atkVsDefense
                ourPokeObject['scoreVs'][theirName] = score
                ourPokeObject['actualMovesVs'][theirName] = actualMovesVs


In [33]:
def chooseRankingForEachLeaderPokemon(availablePokemonObjects, leaderPokemonData):
    bestTeamVsTheirPoke = {}
    for theirName in leaderPokemonData:
        theirPoke = leaderPokemonData[theirName]
        bestTeamVsTheirPoke[theirName] = []
        for ourName in availablePokemonObjects:
            ourPoke = availablePokemonObjects[ourName]
            bestTeamVsTheirPoke[theirName].append(ourPoke)
        bestTeamVsTheirPoke[theirName] = sorted(bestTeamVsTheirPoke[theirName], key=lambda d: d['scoreVs'][theirName],reverse=True) 
    return bestTeamVsTheirPoke

In [47]:
## RUN THIS AFTER RUNNING SETTINGS 

availablePokemonObjects = createAvailablePokemonWithAvailableMoves(encounterable_pokemonParsed, AVERAGE_LEVEL, arena1_Options)
appendMovesVsToPokemonObject(availablePokemonObjects, leaderPokemonData)
appendMoveScoreToPokemonObject(availablePokemonObjects, leaderPokemonData)
final = chooseRankingForEachLeaderPokemon(availablePokemonObjects, leaderPokemonData)

In [52]:
for leaderPokeName in final:
    ourRanking = final[leaderPokeName]
    print('--------------')
    print('ranking for: ', leaderPokeName)
    

    lowestScoreVs = {}
    lowestScoreVs[leaderPokeName] = ourRanking[-1]['scoreVs'][leaderPokeName] * -1
    
    
    highestScoreVs = {}
    highestScoreVs[leaderPokeName] = ourRanking[0]['scoreVs'][leaderPokeName] + lowestScoreVs[leaderPokeName]
    
    for poke in ourRanking:
        print('   ', poke['name'], 'score: ', poke['scoreVs'][leaderPokeName])
        print('      with moves:', poke['actualMovesVs'][leaderPokeName])
        poke['percentScoreVs'] = {}
        poke['percentScoreVs'][leaderPokeName] = (poke['scoreVs'][leaderPokeName] + lowestScoreVs[leaderPokeName]) / highestScoreVs[leaderPokeName]
        
        print('      percentScore:',  poke['percentScoreVs'][leaderPokeName])
    
       # print(poke['movesVs'][leaderPokeName])

--------------
ranking for:  rhyhorn
    omastar score:  28.54166666666667
      with moves: ['water-gun', 'surf', 'bubble-beam', 'water-gun']
      percentScore: 1.0
    slowbro score:  23.26666666666667
      with moves: ['water-gun', 'surf', 'bubble-beam', 'water-gun']
      percentScore: 0.9248664688427299
    articuno score:  22.333333333333332
      with moves: ['ice-beam', 'bubble-beam', 'water-gun', 'ice-beam']
      percentScore: 0.9115727002967358
    golduck score:  22.22357723577236
      with moves: ['surf', 'bubble-beam', 'water-gun', 'ice-beam']
      percentScore: 0.9100094086994281
    seadra score:  21.410256410256412
      with moves: ['bubble', 'water-gun', 'surf', 'bubble-beam']
      percentScore: 0.898425017119379
    omanyte score:  20.8125
      with moves: ['water-gun', 'surf', 'bubble-beam', 'water-gun']
      percentScore: 0.8899109792284865
    lapras score:  19.666666666666668
      with moves: ['water-gun', 'ice-beam', 'surf', 'bubble-beam']
      percent

In [2]:
import numpy as numpy

numpy.interp(256,[1,512],[5,10])

7.495107632093934